In [15]:
import json
import pandas as pd
from datetime import date
from cafu.metadata import path

In [16]:
path_datalake = path('datalake')

In [17]:
r = open(path_datalake+'/metadata.json', 'r')
metadata_datalake = json.load(r)

In [18]:
campeonatos = [
               [k1, k2] for k1 in list(metadata_datalake['jogos_ids'].keys()) 
                        for k2 in list(metadata_datalake['jogos_ids'][k1].keys())
                        if metadata_datalake['jogos_ids'][k1][k2] != 'failed'
              ]

In [ ]:
today = date.today()

In [49]:
prox_partidas = {c:{} for c in metadata_datalake['jogos_ids']}
for c in campeonatos:
    df = pd.read_csv(path_datalake+f'/jogos_ids/{c[0]}/{c[1]}.csv')
    if len(df[df['dates']>=str(today)]['dates'])==0:
        prox_partidas[c[0]][c[1]] = 'finalizado'
    else:
        prox_partidas[c[0]][c[1]] = min(df[df['dates']>=str(today)]['dates'])

In [50]:
prox_partidas

{'brasil': {'2021-2021': 'finalizado',
  '2020-2020': 'finalizado',
  '2019-2019': 'finalizado',
  '2018-2018': 'finalizado',
  '2017-2017': 'finalizado',
  '2016-2016': 'finalizado',
  '2015-2015': 'finalizado',
  '2014-2014': 'finalizado'},
 'espanha': {'2021-2022': '2022-03-11',
  '2020-2021': 'finalizado',
  '2019-2020': 'finalizado',
  '2018-2019': 'finalizado',
  '2017-2018': 'finalizado',
  '2016-2017': 'finalizado',
  '2015-2016': 'finalizado',
  '2014-2015': 'finalizado',
  '2013-2014': 'finalizado'}}

In [38]:
c = ['brasil', '2014-2014']
df = pd.read_csv(path_datalake+f'/jogos_ids/{c[0]}/{c[1]}.csv')
min(df[df['dates']>=str(today)]['dates'])

ValueError: min() arg is an empty sequence

In [1]:
from cafu import path

In [2]:
path('logs_cafu')

'/home/luan/futebol/logs_cafu/2022-03-10'